In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image

Image.open("/kaggle/input/plant-seedlings-classification/test/86f08e6d1.png")

In [ ]:
pd.options.display.max_colwidth = 500

import glob

path = glob.glob("/kaggle/input/plant-seedlings-classification/train/*/*")   

train = pd.DataFrame({"path":path})

train["answer"] = train["path"].apply(lambda x : x.split("/")[5])
train

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid = train_test_split(train, test_size = 0.2, random_state = 1, stratify = train["answer"])


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# 이번 대회는 데이터가 적어서 늘려줘야한다. (4750개)

# data augmentaion (데이터 증강)
# 1. 강아지 사진이 왼쪽을 보고있으면, 위,아래,오른쪽을 보는 강아지 사진은 구분을 잘 못하는 것을 해소해준다.
# 2. 과대적합 방지

idg = ImageDataGenerator(horizontal_flip = True)
# 1. horizontal_flip옵션 : 좌우 반전, epochs마다 50%확률로 좌우반전된 데이터를 학습시킨다. -> epochs를 최소 2번이상은 해줘야함, 일반적으로 도움됨
# 2. vertical_flip옵션 : 상하 반전
# *주의 : 둘다 적용시키는 것은 지양 / 상하,좌우반전 옵션을 둘 다 넣었다가 둘 다 적용되면 데이터가 너무 변형되서 안 좋아 질 수도 있다.
#        각각 하나씩 적용시킨 모델을 앙상블하면 좋다.

train_generator = idg.flow_from_dataframe(x_train, x_col = "path", y_col = "answer", batch_size = 256 , target_size = (100,100))
valid_generator = idg.flow_from_dataframe(x_valid, x_col = "path", y_col = "answer", batch_size = 256 , target_size = (100,100))


In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet
    
# efficientnet모델 : dropout, conv, maxpooling등.. 잘 쌓여있다.

In [ ]:
# eff = EfficientNetB0(weights = "imagenet", include_top = False, pooling = "avg")
# eff.summary()

In [ ]:
from efficientnet.tfkeras import EfficientNetB0 
from keras import Sequential
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau

rl = ReduceLROnPlateau(patience = 2, factor = 0.1, verbose = 1)

model = Sequential()

model.add(EfficientNetB0(weights = "imagenet", include_top = False, pooling = "avg"))
# weight : 가중치
# include_top : 출력층은 알아서 한다. False
# pooling : 일반적으로 global_avg_pooling, 
#           출력층은 보통 Dense layer인데 1차원이라 연결이 안됨, 근데 이걸 Flatten()으로 연결하면 지엽적인 정보 날아감

model.add(Dense(12, activation = "softmax"))

model.compile(metrics = ["acc"], loss = "categorical_crossentropy" , optimizer = "adam")

model.fit(train_generator, validation_data = valid_generator, epochs = 5, callbacks = [rl])
# model.fit(train_generator, validation_data = valid_generator, epochs = 4, callbacks = [rl])

# 개선
# 1. 이미지 사이즈 증가 
# 2. learing_rate 줄이기

In [ ]:
test = pd.DataFrame({"path":os.listdir("/kaggle/input/plant-seedlings-classification/test")})
test["path"] = "/kaggle/input/plant-seedlings-classification/test/"+test["path"]
test

idg2 = ImageDataGenerator()
# test time augmentation기법도 있다
test_generator = idg2.flow_from_dataframe(test,x_col = "path", y_col = None, batch_size = 128, target_size=(100,100), class_mode = None, shuffle = False)

result = model.predict(test_generator)

In [ ]:
sub = pd.read_csv("/kaggle/input/plant-seedlings-classification/sample_submission.csv")
sub.head()

In [ ]:
result = result.argmax(1)
category = sorted(train["answer"].unique())

a = [category[result[i]] for i in range(len(test))]

In [ ]:
sub["species"] = a
sub.head()

In [ ]:
sub["file"] = test_gen.filenames
sub["file"] = sub["file"].apply(lambda x : x.split("/")[5])

In [ ]:
sub.to_csv("sub.csv", index = False)